# 📡 2.45GHz Patch Antenna 設計工具 (Gradio 版)
此 Colab 筆記本使用 Gradio 建立互動式 GUI，使用者可直接輸入參數並觀察天線結構圖與設計參數。

In [ ]:
# ✅ 安裝必要套件
!pip install gradio matplotlib

In [ ]:
# ▶️ 執行 Gradio App
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt

def patch_design(f_r, er, h, t):
    c = 3e8
    f_r_hz = f_r * 1e9
    h_m = h / 1000
    t_m = t / 1000

    W = c / (2 * f_r_hz * np.sqrt((er + 1) / 2))
    er_eff = (er + 1)/2 + (er - 1)/2 * (1 / np.sqrt(1 + 12 * h_m / W))
    delta_L = h_m * 0.412 * ((er_eff + 0.3)*(W/h_m + 0.264)) / ((er_eff - 0.258)*(W/h_m + 0.8))
    L_eff = c / (2 * f_r_hz * np.sqrt(er_eff))
    L = L_eff - 2 * delta_L

    Q = (1 / (np.sqrt(er_eff) - 1)) * (L / h_m) * (1 + (1 / (W / h_m)))
    BW_frac = 1.5 / Q
    BW = BW_frac * f_r_hz / 1e6

    fig, ax = plt.subplots()
    ax.add_patch(plt.Rectangle((0, 0), W*1000, L*1000, edgecolor='black', facecolor='lightblue'))
    ax.text(W*500, L*500, 'Patch', ha='center', va='center')
    ax.set_xlim(-5, W*1000 + 5)
    ax.set_ylim(-5, L*1000 + 5)
    ax.set_aspect('equal')
    ax.set_title('Patch Antenna 結構圖')
    plt.xlabel('寬度 W (mm)')
    plt.ylabel('長度 L (mm)')
    plt.grid(True)

    return (
        f"{W*1000:.2f} mm",
        f"{er_eff:.4f}",
        f"{L_eff*1000:.2f} mm",
        f"{L*1000:.2f} mm",
        f"{BW:.2f} MHz",
        fig
    )

iface = gr.Interface(
    fn=patch_design,
    inputs=[
        gr.Number(label="中心頻率 f_r (GHz)", value=2.45),
        gr.Number(label="介電常數 ε_r", value=4.4),
        gr.Number(label="基板厚度 h (mm)", value=1.6),
        gr.Number(label="銅箔厚度 t (mm)", value=0.034)
    ],
    outputs=[
        gr.Textbox(label="貼片寬度 W"),
        gr.Textbox(label="有效介電常數 ε_eff"),
        gr.Textbox(label="貼片有效長度 L_eff"),
        gr.Textbox(label="貼片實際長度 L"),
        gr.Textbox(label="頻寬估算 BW"),
        gr.Plot(label="Patch 天線圖")
    ],
    title="📡 Patch Antenna 設計工具",
    description="輸入參數後，自動估算貼片尺寸與頻寬，並顯示天線結構圖"
)

iface.launch(share=True)